# Introduction : Legal AI agent 

The Legal AI Agent is designed to help users understand and analyze legal situations. It interprets user queries, identifies key legal elements, and retrieves relevant information from the French Labor Code (in this demo, it can be extended to other areas of law) using a RAG (Retrieval-Augmented Generation) system. When necessary, it performs web searches via Serper to find official sources. Finally, it provides clear, well-reasoned answers with legal references and links to trusted resources.

## Configure Bedrock

Create the necessary clients to invoke Bedrock models. If you would need to pass in a certain role then set those values appropriately
We begin with instantiating the LLM and the Embeddings model. Here we are using Mistralsmall for text generation and Titan Embeddings G1 - Text for text embeddings.



In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import json
import os
import sys
import boto3
import botocore


from langchain.llms.bedrock import Bedrock

ModuleNotFoundError: No module named 'boto3'

In [ ]:

# Ajout d'un module utilitaire (optionnel)
module_path = ".."
sys.path.append(os.path.abspath(module_path))


# ---- ⚠️ Config AWS ----
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
# os.environ["AWS_PROFILE"] = ""
# os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:..."  # si nécessaire


boto3_bedrock = boto3.client(
    "bedrock",
    region_name=os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
)


bedrock_runtime = boto3.client(
    service_name= "bedrock-runtime",
    region_name=os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
)

# ---- Paramètres spécifiques Mistral ----
model_parameters = {
    "temperature": 0.0,
    "top_p": 0.5,
    "top_k": 200,
    "max_tokens": 200,               # ✅ Utilisé pour Mistral
    
}

# ---- Initialisation du LLM Mistral via LangChain ----
llm = Bedrock(
    model_id="mistral.mistral-small-2402-v1:0",
    model_kwargs=model_parameters,
    client=bedrock_runtime
)



/tmp/ipykernel_3303/1697123187.py:33: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockLLM``.
  llm = Bedrock(


## Implementation
In order to follow the approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

 - **LLM (Large Language Model)**: Mistral Small via Amazon Bedrock
Used to interpret document chunks and generate human-friendly answers.

 - **Embeddings Model**: Amazon Titan Embeddings via Amazon Bedrock
Used to generate numerical representations of textual documents.

 - **Document Loader**: S3FileLoader
 and PDF Loader available through LangChain.

 - **PyPDF2**: Used to read PDF files and extract their text.
For this notebook, we load a sample file from a local path, but in production, this can easily be replaced by a loader that processes the full corpus of legal texts.

  - **Vector Store**: FAISS (in-memory)
Used to compare input embeddings with document embeddings to retrieve the most relevant sections.

 - **Wrapper**: Combines the index, vector store, embeddings model, and LLM to abstract the underlying logic from the user.

 - **ConversationalRetrievalChain.from_llm**: Creates a conversational retrieval chain using ConversationBufferMemory for context persistence — all from LangChain.

 - **SERPER_API_KEY**: Used to perform web searches and retrieve official links when additional context is needed.

 - **initialize_agent**: Used to create the intelligent legal agent. 


## Extract `product relevant` information

Les attributs à extraire sont :
  - domaine_du_droit (ex : droit du travail, droit civil, droit pénal)
  - type_de_problème (ex : licenciement, contrat, harcèlement)
  - statut_de_l_utilisateur (ex : salarié, employeur, fonctionnaire)
  - mots_cles (liste de mots ou expressions clés liés au problème)


We will instruct LLM to fetch the relevant information from the user prompt based on the above so we can query our catalog and bring back relevant results. Note we have a very specific prompt template for the `entity extraction`

In [ ]:
ner_prompt = """[INST]
Analyse la question de l'utilisateur et identifie les éléments suivants.

### Instructions :
- Extrais les attributs pertinents :
  - domaine_du_droit (ex : droit du travail, droit civil, droit pénal)
  - type_de_problème (ex : licenciement, contrat, harcèlement)
  - statut_de_l_utilisateur (ex : salarié, employeur, fonctionnaire)
  - mots_cles (liste de mots ou expressions clés liés au problème)

- Si tu ne trouves pas d’attribut, omets-le.
- Réponds uniquement en JSON entre <attributes></attributes>.
- N’ajoute pas de texte explicatif.
- Réponds en français.

### Entrée :
{customer_input}

### Réponse :
[/INST]
"""

entity_extraction_result = llm(ner_prompt.format(customer_input=customer_input)).strip()
print(entity_extraction_result)


<attributes>
{
  "domaine_du_droit": "droit du travail",
  "statut_de_l_utilisateur": "salarié",
  "type_de_problème": "licenciement",
  "partie_adverse": "employeur",
  "juridiction_potentielle": "conseil de prud'hommes",
  "urgence_ou_nature": "contentieuse"
}
</attributes>


In [ ]:
ner_prompt = """[INST]
Analyse la situation juridique décrite par l'utilisateur et extrais les éléments nécessaires pour une analyse de cas et une recherche documentaire.

### Instructions (obligatoires) :
- Retourne **uniquement** un objet JSON entre les balises <attributes></attributes>.
- Ne fournis **aucune** explication textuelle en dehors des balises.
- Réponds **en français**.
- Normalise les dates au format ISO (YYYY-MM-DD) lorsque c'est possible.
- Les montants doivent être numériques (séparer la devise si précisé, ex: {"montant": 1200, "devise":"EUR"}).
- Si un champ n'est pas présent dans l'entrée, **omets** ce champ du JSON.

### Champs à extraire (nom exact des clés) :
- domaine_du_droit : string (ex: "droit du travail")
- type_de_probleme : string (ex: "licenciement", "non-paiement de salaire")
- statut_de_l_utilisateur : string (ex: "salarié", "locataire")
- partie_adverse : string (ex: "employeur", "bailleur", "État")
- date_evenement : ISO date (si connue)
- date_notification : ISO date (date de la lettre/notification si connue)
- montants : objet { "montant": nombre, "devise": string } (si applicable)
- faits_cles : liste de courtes chaînes (les points factuels essentiels, 3-6 items max)
- pieces_justificatives : liste de noms de documents mentionnés / uploadés (ex: "contrat.pdf", "courrier_employeur.pdf")
- juridiction_potentielle : string (ex: "prud’hommes", "tribunal judiciaire")
- urgence : string parmi ["urgente", "à traiter sous 1 mois", "non urgente"] (si déductible)
- gravite : string parmi ["faible","moyenne","élevée"] (estimation succincte)
- objectif_utilisateur : string (ex: "annulation du licenciement", "remboursement", "indemnisation")
- recommandations_preliminaires : liste courte (1-3 actions pratiques immédiates, ex: "envoyer mise en demeure", "collecter fiches de paie")
- confidence : nombre entre 0 et 1 (estimation de la confiance du modèle dans l'extraction)

### Exemple d'entrée (inclus ci-dessous pour le modèle) :
{customer_input}

### Réponse :
[/INST]

"""

entity_extraction_result = llm(ner_prompt.format(customer_input=customer_input)).strip()
print(entity_extraction_result)

#### Extract values into JSON

Since we have instructed LLM to return our data as XML wrapping a JSON, we run the necessary extraction steps to fetch the relevant details.

In [ ]:
import re
import json
result = re.search('<attributes>(.*)</attributes>', entity_extraction_result, re.DOTALL)
attributes = json.loads(result.group(1))
attributes

{'domaine_du_droit': 'droit du travail',
 'statut_de_l_utilisateur': 'salarié',
 'type_de_problème': 'licenciement',
 'partie_adverse': 'employeur',
 'juridiction_potentielle': "conseil de prud'hommes",
 'urgence_ou_nature': 'contentieuse'}

In [ ]:
pip install unstructured

Note: you may need to restart the kernel to use updated packages.


## Use Retrieval Augmented Generation (RAG) 

We will leverage the semantic search to embed product-accessory catalogs and order history fro the embeddings created by `Amazon Titan Embeddings Text v1`

After downloading we can load the documents with the help of [S3FileLoader available under LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 8k tokens, which roughly translates to ~32000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

Here we are fetching our product catalog and creating the embeddings for 
1. a part of the french work law , (For Demo)
we can later use the expanded dataset .

In [ ]:
#%pip install --upgrade "unstructured[pdf]"


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install PyPDF2


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import boto3
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
import numpy as np

# --- Configuration S3 ---
bucket = "sagemaker-us-east-1-745039059599"
key = "data_juridique/codes/LEGITEXT000006072050.pdf"
local_path = "/tmp/LEGITEXT000006072050.pdf"

s3 = boto3.client("s3", region_name="us-east-1")
s3.download_file(bucket, key, local_path)

# --- Lecture PDF partielle ---
reader = PdfReader(local_path)
text = ""

# Exemple : lire les pages 5 à 15 (Python compte à partir de 0)
for i in range(5, 200):
    if i < len(reader.pages):
        page = reader.pages[i]
        text += page.extract_text() or ""  # éviter None si une page est vide

# --- Créer un document LangChain ---
documents = [Document(page_content=text, metadata={"source": key})]

# --- Split en chunks ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(documents)

In [ ]:
# --- Embedding (Amazon Titan via Bedrock) ---
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    client=bedrock_runtime
)

vectorstore_faiss = FAISS.from_documents(split_docs, bedrock_embeddings)
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

# --- Test de requête ---
customer_input = "Quels sont les droits du salarié en cas de licenciement économique ?"
query_embedding = bedrock_embeddings.embed_query(customer_input)
print(np.array(query_embedding).shape)


/tmp/ipykernel_3303/320951671.py:2: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  bedrock_embeddings = BedrockEmbeddings(


(1024,)


In [ ]:
retriever = wrapper_store_faiss.vectorstore.as_retriever(search_kwargs={"k": 3})


# Conversational RAG

In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


/tmp/ipykernel_3303/4063055825.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [55]:
from langchain.chains import ConversationalRetrievalChain

rag_chat = ConversationalRetrievalChain.from_llm(
    llm=llm,          # ton LLM Mistral ou Claude sur Bedrock
    retriever=retriever,
    memory=memory
)


The ask_rag() function extracts key legal entities from a user’s question and builds a context-enriched query. It then uses a RAG model to retrieve relevant legal information and generate a concise answer in French.
we will use it to interact with the RAG model.

In [54]:
def ask_rag(question, llm, ner_prompt, rag_chat):
    
    # Extraire NER
    entity_json = llm(ner_prompt.format(customer_input=question)).strip()
    result = re.search('<attributes>(.*)</attributes>', entity_json, re.DOTALL)
    attributes = json.loads(result.group(1))
    
    # Construire requête enrichie
    query = f"""
    Domaine du droit : {attributes.get('domaine_du_droit', '')}
    Type de problème : {attributes.get('type_de_problème', '')}
    Statut : {attributes.get('statut_de_l_utilisateur', '')}
    partie_adverse: {attributes.get('partie_adverse', '')}
    juridiction_potentielle: {attributes.get('juridiction_potentielle', '')}
    
    
    Question : {question}
    Réponds **en français**, de manière claire et concise.

    """
    
    # RAG
    return rag_chat.run(query)


In [42]:
question1 = "Quels sont les droits du salarié en cas de licenciement économique ?"
response1 = rag_chat.run(question1)
print("Réponse 1 :", response1)

question2 = "Et si l'employé est en période d'essai ?"
response2 = rag_chat.run(question2)
print("Réponse 2 :", response2)




Réponse 1 :  The sector of activity is used to assess the economic cause of the dismissal. It is characterized by the nature of the products, goods, or services provided, the targeted clientele, as well as the networks and distribution methods related to the same market.

The employee also has the right to reclassification efforts by the employer before the economic dismissal can take place. The employer must communicate all the elements related to the dismissal to the judge in case of a dispute.

In companies with at least 50 employees, if the dismissal project concerns at least 10 employees in the same period of 30 days and the dismissal occurs without any decision on validation or homologation, or if a negative decision has been made, the dismissal is null.
Réponse 2 :  The provided text does not specifically address the rights of an employee during their probationary period in the case of an economic dismissal. However, it's important to note that the rules for economic dismissal g

In [56]:
question1 = "Quels sont les droits du salarié en cas de licenciement économique ?"
response1 = ask_rag(question1, llm, ner_prompt, rag_chat)



In [57]:
print(response1)

 In case of economic dismissal, the rights of an employee during the trial period are similar to those of an employee outside the trial period. The employee is entitled to re-employment priority and to efforts in training and adaptation. However, the employer can terminate the employment contract without having to respect the notice period or pay severance pay. If the dismissal is deemed abusive, the employee can seize the labor court to request damages and interest.

In the provided context, there is no specific mention of the rights of an employee during the trial period in case of economic dismissal. However, based on the general principles of the French Labor Code, the above answer can be inferred.

Reference(s):
- Code du travail - Dernière modification le 01 octobre 2025 - Document généré le 17 octobre 2025
- Article L1233-4, L1233-6


In [58]:
question2 = "Et si l'employé est en période d'essai ?"
response2 = ask_rag(question2, llm, ner_prompt, rag_chat)
print(response2)

 In the context provided, the text does not mention the rights of an employee during the trial period in case of economic dismissal. Therefore, I cannot provide a specific answer based on the information given. However, in general, the rights of an employee during the trial period are often more limited than those of a permanent employee. It is important to consult the specific labor code or seek legal advice for accurate information.


In [61]:
question3 = "what is the source of ur answers"
response3 = ask_rag(question3, llm, ner_prompt, rag_chat)
print(response3)

 The source of my answers is the French Labor Code, specifically the articles L1221-1, L1221-2, L1221-3, and the section related to the formation of the employment contract. Additionally, the articles L1251-61 and L1251-62 regarding temporary work agencies and public legal entities are also referenced.


In [62]:
question4 = "can u give the links for these articles"
response4 = ask_rag(question4, llm, ner_prompt, rag_chat)
print(response4)

 The provided text does not contain information about the rights of a salarié (employee) during the period of trial (période d'essai) in case of economic layoff (licenciement économique). Therefore, I cannot provide a specific answer based on the given context.

However, in general, during the period of trial, the employer and employee can terminate the employment contract without cause or notice. This is known as the "right to resign" (droit de démission) for the employee and the "right to dismiss" (droit de licenciement) for the employer.

If the employer decides to terminate the contract during the period of trial for economic reasons, the employee would not be entitled to the same rights as a regular employee who has completed their period of trial. These rights typically include a notice period, severance pay, and priority for reemployment.

For a more accurate and detailed response, I


# Agent

In [63]:
from langchain.chains import ConversationalRetrievalChain
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
import requests, os, json


Search in the official web sites , using SERPER_API_KEY 

In [ ]:
def search_serper(query):
    SERPER_API_KEY = "421506e493d610be99e040655d3fa681f6c92f69"  
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}

    # On restreint la recherche à des sites officiels
    payload = {"q": query + " site:legifrance.gouv.fr OR site:service-public.fr OR site:travail-emploi.gouv.fr"}

    res = requests.post(url, headers=headers, json=payload)
    if res.status_code != 200:
        return f"Erreur lors de la recherche : {res.text}"

    data = res.json()
    links = [r["link"] for r in data.get("organic", [])[:3]]
    #return "\n".join(links) if links else "Aucun lien officiel trouvé."
    print("\n".join(links) if links else "Aucun lien officiel trouvé.")

In [76]:
search_serper("étranger")

https://www.service-public.fr/particuliers/vosdroits/N105
https://www.service-public.fr/particuliers/vosdroits/N19804
https://www.service-public.fr/particuliers/vosdroits/F11201


Defining the tools from where the agent has to choose to answer the questions.

In [ ]:
tools = [
    Tool(
        name="Recherche juridique locale",
        func=lambda q: rag_chat.run(q),
        description="Utilise la base locale (Code du travail, lois) pour répondre aux questions juridiques."
    ),
    Tool(
        name="Recherche web officielle",
        func=search_serper,
        description="Recherche des liens officiels (Legifrance, Service-Public) liés à la question."
    )
]

In [67]:
memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    memory=memory,
    verbose=True
)


/tmp/ipykernel_3303/3637044846.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


The same way we did for the conversational agent to automate interactions with it, we will implement the ask_agent function, which performs the same steps.

In [ ]:
def ask_agent(user_input, llm, ner_prompt, agent):
    """Combine extraction d'entités, enrichissement de la requête, et raisonnement."""
    
    # Étape 1 : Extraction des attributs juridiques
    extraction = llm(ner_prompt.format(customer_input=user_input)).strip()

    # Extraction JSON
    try:
        attributes_text = extraction.split("<attributes>")[1].split("</attributes>")[0]
        attributes = json.loads(attributes_text)
    except Exception:
        attributes = {}

    # Étape 2 : Construction de la requête enrichie
    query = f"""
    Domaine du droit : {attributes.get('domaine_du_droit', '')}
    Type de problème : {attributes.get('type_de_problème', '')}
    Statut : {attributes.get('statut_de_l_utilisateur', '')}
    Mots-clés : {', '.join(attributes.get('mots_cles', [])) if isinstance(attributes.get('mots_cles'), list) else attributes.get('mots_cles', '')}

    Question : {user_input}
    """

    # Étape 3 : Exécution via l’agent (il choisira le bon outil)
    response = agent.run(query)
    return response








In [ ]:
question1 = "Quels sont les droits du salarié en cas de licenciement économique ?"
response1 = ask_agent(question1, llm, ner_prompt, agent)
print("\n🧩 Réponse 1 :", response1)





> Entering new AgentExecutor chain...
 The user is asking about the rights of an employee in case of economic dismissal. I should use the local legal research tool to find information in the Labor Code and relevant laws.

Action: Recherche juridique locale
Action Input: droits salarié licenciement économique

Observation:  In case of economic dismissal, the rights of an employee on a trial period are similar to those of an employee outside the trial period. However, the employer can terminate the employment contract without respecting the notice period and without paying a dismissal allowance. The employee is also entitled to efforts in training and adaptation, as well as to rehiring priorities. If the company has carried out economic dismissals of more than ten employees in total over three months, the employee on a trial period is also entitled to a plan for safeguarding employment.

Source:

* Article L1233-3 of the French Labor Code: "The employer may terminate the employment con

In [ ]:
question2 = "Et si l'employé est en période d'essai ?"
response2 = ask_agent(question2, llm, ner_prompt, agent)
print("\n🧩 Réponse 2 :", response2)





> Entering new AgentExecutor chain...
 The question is asking about the rights of an employee during their probationary period. I should use the local legal research tool to find information about this in the French labor code.

Action: Recherche juridique locale
Action Input: période d'essai, droits de l'employé

Observation: 

The text provided does not mention the rights of an employee during the trial period in case of economic dismissal. However, according to the French Labor Code, the employer can terminate the employment contract during the trial period without respecting the notice period and without paying a dismissal allowance. The employee is also entitled to efforts in training and adaptation, as well as to rehiring priorities. If the company has carried out economic dismissals of more than ten employees in total over three months, the employee on a trial period is also entitled to a plan for safeguarding employment.

Source:

* Article L1233-3 of the French Labor Code: "

In [ ]:
question3 = "Peux-tu me donner les liens officiels correspondants ?"
response3 = ask_agent(question3, llm, ner_prompt, agent)
print("\n🔗 Réponse 3 :", response3)



> Entering new AgentExecutor chain...
 The user is asking for official links related to some legal topic, but they didn't specify which one. I need to clarify this first.
Action: None
Action Input: None
Observation: None is not a valid tool, try one of [Recherche juridique locale, Recherche web officielle].
Thought: I need to ask the user to specify the legal topic they're interested in.
Action: None
Action Input: None
Observation: None is not a valid tool, try one of [Recherche juridique locale, Recherche web officielle].
Thought:

ValueError: Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again.

In [ ]:
question3 = " je veux prendre un rendez vous pour changer mon titre de séjour à cergy"
response3 = ask_agent(question3, llm, ner_prompt, agent)
print("\n Réponse 3 :", response3)

KeyboardInterrupt: 

# END

Thank you for reading .